In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# # This is to copy the files from Drive to the SSD for runtime to make training quicker.
!pwd
!mkdir -p /content/cs7643_model_quantization/data
!cp -r "/content/drive/MyDrive/cs7643_model_quantization/data/gtFine_trainId" "/content/cs7643_model_quantization/data"
!cp -r "/content/drive/MyDrive/cs7643_model_quantization/data/gtFine_trainIdColorized" "/content/cs7643_model_quantization/data"
!cp -r "/content/drive/MyDrive/cs7643_model_quantization/data/leftImg8bit_trainvaltest.zip" "/content/cs7643_model_quantization/data"
!unzip -o /content/cs7643_model_quantization/data/leftImg8bit_trainvaltest.zip -d /content/cs7643_model_quantization/data/leftImg8bit_trainvaltest
!rsync -av --exclude='data' --exclude='.*' --exclude='testing.ipynb' --exclude='myvenv' /content/drive/MyDrive/cs7643_model_quantization/ /content/cs7643_model_quantization/


In [ ]:
%cd /content/cs7643_model_quantization/
!pwd

In [ ]:
!python -m pipeline.fine_tuning

In [ ]:
!python -m src.quantization.qat

In [ ]:
import importlib
from src.quantization import qat

importlib.reload(qat)


In [ ]:
import importlib
from src.quantization import quantization_utils

importlib.reload(quantization_utils)

In [6]:
import importlib
from pipeline import create_dataset

importlib.reload(create_dataset)

<module 'pipeline.create_dataset' from '/Users/catherinedemmerle/Desktop/GaTech/DL/assignments/final_project/cs7643_model_quantization/pipeline/create_dataset.py'>

In [ ]:
import os
import glob
import yaml
import itertools
import copy
from src.quantization.quantization_utils import build_qconfig
from pipeline.create_dataset import cityScapesDataset
from src.quantization.qat import run_qat

results_dir = "/content/drive/MyDrive/cs7643_model_quantization/qat_results_int8_fix/"
# results_dir = "qat_results_test"
os.makedirs(results_dir, exist_ok=True)

# Load configuration.
with open("configs/ptq.yaml", "r") as f:
    config = yaml.safe_load(f)["qat"]

# configurable params
modes = ['int8']
act_observers = ["minmax", "histogram", "movingavg"]
training_batch_sizes = [8] #, 8, 16]
training_learning_rates = [1e-4, 1e-5]
skip_aspps = [False, True]

for i, (mode, act_observer, training_batch_size, training_learning_rate, skip_aspp) in enumerate(itertools.product(modes, act_observers, training_batch_sizes, training_learning_rates, skip_aspps)):
  current_config = copy.deepcopy(config)
  current_config['mode'] = mode
  current_config["activations"]["observer"] = act_observer
  current_config["training"]["batch_size"] = training_batch_size
  current_config["training"]["learning_rate"] = training_learning_rate
  current_config["skip_aspp"] = skip_aspp

  print(f"Run {i} with config: {current_config}")

  run_qat(i, current_config, results_dir)
